In [1]:
SOURCE_STRING = """
This is AI News! an MVP of a service that goes thru all AI discords/Twitters/reddits and summarizes what people are talking about, so that you can keep up without the fatigue. Signing up here opts you in to the real thing when we launch it 🔜

a quiet day is all you need.

AI News for 9/25/2024-9/26/2024. We checked 7 subreddits, 433 Twitters and 31 Discords (224 channels, and 3282 messages) for you. Estimated reading time saved (at 200wpm): 342 minutes. You can now tag @smol_ai for AINews discussions!

Many are still processing the surprise management turnover at OpenAI. Sama and gdb both posted statements. It seems the Anthropic rumors were postponed, but in the meantime the new blueberry model rumor mill is just getting started.

Since it's a quiet day, you could help out AINews by checking out the RAG++ course from Weights and Biases! We featured it yesterday but forgot to include the text link. Sorry!

Swyx: Something we also missed in our initial scan yesterday was chapters 6 and 7 on response synthesis and optimmization. Chapter 6 in particular is exactly what we had to do to build AINews - everything you see below is AI generated thanks to these techniques.
"""




In [2]:
STORYBOARD_PROMPT = """
You are a helpful assistant that takes a source string of news and exciting events, and returns a storyboard for a short video with content about the news. 

Each storyboard item should contain a description of a starting image (which we will generate based on your description), a description of the ending image (which we will also generate based on your description), and a description of the action that should happen in between

SOURCE STRING:
{source_string}
"""


In [13]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Prepare the prompt
prompt = STORYBOARD_PROMPT.format(source_string=SOURCE_STRING)


class Storyboard(BaseModel):
    start_image_description: str
    end_image_description: str
    action: str

# Call OpenAI API with GPT-4 Turbo
response = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": prompt},
    ],
    response_format=Storyboard
)

# Extract the generated storyboard
storyboard = response.choices[0].message.parsed

print("Generated Storyboard:")
print(storyboard)


Generated Storyboard:
start_image_description='An animated representation of various social media logos (Discord, Twitter, Reddit) floating around a computer screen, symbolizing the gathering of information from these platforms.' end_image_description="The screen transitions to show the AINews logo prominently, with a call-to-action button that says 'Sign Up Now!' appearing below it." action='A narrator introduces AINews, explaining how it collects information from multiple AI-related platforms. The camera zooms in on the screen as it shows snippets of conversations from Discord and Twitter relating to AI news. The narrator highlights the time saved by using AINews for busy individuals who want to keep up with AI developments without the effort of scrolling through numerous posts.'


In [15]:
%reload_ext autoreload
%autoreload 2

import requests
IDEOGRAM_URL = "https://api.ideogram.ai/generate"

IDEOGRAM_HEADERS = {
    "Api-Key": os.getenv("IDEOGRAM_API_KEY"),
    "Content-Type": "application/json"
}
start_image_request = {
    "image_request": {
        "prompt": storyboard.start_image_description,
        "model": "V_2",
        "magic_prompt_option": "AUTO"
    }
}


start_image_response = requests.post(IDEOGRAM_URL, json=start_image_request, headers=IDEOGRAM_HEADERS)
print(start_image_response.json())

end_image_request = {
    "image_request": {
        "prompt": storyboard.end_image_description,
        "model": "V_2",
        "magic_prompt_option": "AUTO"
    }   
}

end_image_response = requests.post(IDEOGRAM_URL, json=end_image_request, headers=IDEOGRAM_HEADERS)
print(end_image_response.json())



{'created': '2024-09-27T00:06:52.666768+00:00', 'data': [{'is_image_safe': True, 'prompt': 'An animated representation of various social media logos (Discord, Twitter, Reddit) floating around a computer screen, symbolizing the gathering of information from these platforms.', 'resolution': '1024x1024', 'seed': 1626649855, 'style_type': 'GENERAL', 'url': 'https://ideogram.ai/api/images/ephemeral/_vopFDt_R3G9eFB7IMECKA.png?exp=1727482032&sig=d3ce83dc1d177674531ebe541fb3db0821ae133168145471df324a66fb0ffb46'}]}
{'created': '2024-09-27T00:07:12.264545+00:00', 'data': [{'is_image_safe': True, 'prompt': "The screen transitions to show the AINews logo prominently, with a call-to-action button that says 'Sign Up Now!' appearing below it.", 'resolution': '1024x1024', 'seed': 429542606, 'style_type': 'GENERAL', 'url': 'https://ideogram.ai/api/images/ephemeral/CVoVPjIESj6xoxoknYoTIw.png?exp=1727482051&sig=03fdfeebad6898d8e32521689f96d64a29e42de847bf974357b81721c11e5f72'}]}


In [16]:
from lumaai import LumaAI

client = LumaAI()

generation = client.generations.create(
    prompt=storyboard.action,
    keyframes={
      "frame0": {
        "type": "image",
        "url": start_image_response.json()["data"][0]["url"]
      },
      "frame1": {
        "type": "image",
        "url": end_image_response.json()["data"][0]["url"]
      }
    }
)



KeyError: 'url'